In [2]:
%reset -f

import os
import pandas as pd
import matplotlib.pyplot as plt
import bz2
import ujson




directory = "Z:/04_Daten/GroundThruth/Videos_TUD/"

IOU_files = ['otcamera-tud03_FR20_2023-10-26_08-00-00_IOU.counts_15min.csv',
             'otcamera-tud03_FR20_2023-10-26_08-15-00_IOU.counts_15min.csv',
             'otcamera-tud03_FR20_2023-10-26_08-30-00_IOU.counts_15min.csv',
             'otcamera-tud03_FR20_2023-10-26_08-45-00_IOU.counts_15min.csv',
             'otcamera-tud03_FR20_2023-10-26_09-00-00_IOU.counts_15min.csv',
             'otcamera-tud03_FR20_2023-10-26_09-15-00_IOU.counts_15min.csv',
             'otcamera-tud03_FR20_2023-10-26_09-30-00_IOU.counts_15min.csv',
             'otcamera-tud03_FR20_2023-10-26_09-45-00_IOU.counts_15min.csv',
             ]

EIOU_files = ['otcamera-tud03_FR20_2023-10-26_08-00-00_EIOU.counts_15min.csv',
              'otcamera-tud03_FR20_2023-10-26_08-15-00_EIOU.counts_15min.csv',
              'otcamera-tud03_FR20_2023-10-26_08-30-00_EIOU.counts_15min.csv',
              'otcamera-tud03_FR20_2023-10-26_08-45-00_EIOU.counts_15min.csv',
              'otcamera-tud03_FR20_2023-10-26_09-00-00_EIOU.counts_15min.csv',
              'otcamera-tud03_FR20_2023-10-26_09-15-00_EIOU.counts_15min.csv',
              'otcamera-tud03_FR20_2023-10-26_09-30-00_EIOU.counts_15min.csv',
              'otcamera-tud03_FR20_2023-10-26_09-45-00_EIOU.counts_15min.csv',
              ]

df_IOU = pd.DataFrame()
for file in IOU_files:
    df = pd.read_csv(directory + file)
    df_IOU = pd.concat([df_IOU, df])

    
df_EIOU = pd.DataFrame()
for file in EIOU_files:
    df = pd.read_csv(directory + file)
    df_EIOU = pd.concat([df_EIOU, df])


del file, df

# Import Ground Truth
GT_files = ['otcamera-tud03_FR20_2023-10-26_08-00-00.otgtevents',
            'otcamera-tud03_FR20_2023-10-26_08-15-00.otgtevents']

df_GT = pd.DataFrame()
for file in GT_files:
    with bz2.open(directory + file, "rt", encoding="UTF-8") as file:
        dictfile = ujson.load(file)
        
        SECTIONS = pd.DataFrame.from_dict(dictfile['sections'])
        EVENTS = pd.DataFrame.from_dict(dictfile['event_list'])
        EVENTS = pd.merge(EVENTS, SECTIONS, left_on='section_id', right_on='id')
        enter_events = EVENTS.groupby('road_user_id').first().rename(columns={'name': 'name_enter'}).reset_index()
        exit_events = EVENTS.groupby('road_user_id').last().rename(columns={'name': 'name_exit'}).reset_index()
        
        exit_events = exit_events[['road_user_id', 'name_exit']]
        
        GTEVENTS = pd.merge(enter_events, exit_events, on = 'road_user_id').rename(columns={'road_user_type': 'classification'})
        del enter_events, exit_events
        
        GTEVENTS['flow'] = GTEVENTS['name_enter'] + " --> " + GTEVENTS['name_exit']
        GTEVENTS['start occurrence time'] = GTEVENTS['video_name'].str[-8:]
        GTEVENTS['start occurrence time'] = GTEVENTS['start occurrence time'].str.replace('-',':')
        
        df_GT = pd.concat([df_GT, GTEVENTS])
        
        
del dictfile, SECTIONS, EVENTS, GTEVENTS,

GTCOUNTS = df_GT.groupby(['start occurrence time', 'classification', 'flow'])['type'].count().reset_index().rename(columns={'type': 'GT'})

GTCOUNTS.to_csv(directory + "GT.csv")

# Auswertung
df_IOU = df_IOU[['start occurrence time', 'classification', 'flow', 'count']].rename(columns={'count': 'IOU'})
df_EIOU = df_EIOU[['start occurrence time', 'classification', 'flow', 'count']].rename(columns={'count': 'EIOU'})

COUNTS = pd.merge(df_IOU, df_EIOU, on = ['start occurrence time', 'classification', 'flow'])
COUNTS = pd.merge(COUNTS, GTCOUNTS, on = ['start occurrence time', 'classification', 'flow'])
del df_IOU, df_EIOU



    


